# 📦 COCO to YOLO Segmentation Converter

## Overview
This notebook provides a comprehensive pipeline for extracting custom subsets from the COCO dataset and converting them to YOLO segmentation format. It handles downloading images, converting annotations (both bounding boxes and segmentation polygons), and creating a properly structured YOLO dataset ready for training.

## Features
- **Custom Subset Extraction**: Select specific classes and sample sizes from COCO
- **Multi-Split Support**: Handles both train and validation splits with balancing
- **YOLO Format Conversion**: Converts COCO annotations to YOLO segmentation format
- **Parallel Processing**: Multi-threaded image downloading for faster processing
- **Automatic Dataset Structure**: Creates proper YOLO directory structure with data.yaml
- **Visualization Tools**: Built-in visualization for verifying conversions
- **Duplicate Handling**: Ensures no overlap between train and validation sets


## Configuration
Edit the `my_categories` dictionary to select desired classes and the sampling parameters:
```python
my_categories = {
    "person": 0,
    "car": 1,
    "dog": 2,
    # Add your classes here
}
N_sample_train = 400  # Minimum samples per class for training
N_sample_valid = 100  # Minimum samples per class for validation
```

## Requirements
- COCO annotation files (instances_train2017.json, instances_val2017.json)
- Python packages: `pycocotools`, `matplotlib`, `requests`, `pyyaml`, `progress`
- Sufficient disk space for images

## Usage Instructions
1. **Configure Categories**: Edit `my_categories` dict with desired classes
2. **Set Sample Sizes**: Adjust `N_sample_train` and `N_sample_valid`
3. **Run Extraction**: Execute cells to download and convert dataset
4. **Verify**: Use visualization cells to check conversions
5. **Download**: Zip file is created automatically (Google Colab)

---

In [ ]:
!pip install progress
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import numpy as np
import yaml
import os
import requests
import shutil
import time
from pathlib import Path
import os
from multiprocessing import Pool, cpu_count
from functools import partial
from progress.bar import Bar

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

#cocoTrain=COCO('/content/instances_train2017.json')
#cocoVal=COCO('/content/instances_val2017.json')

In [ ]:
def COCONames_to_COCOId(coco, COCO_name):
    return coco.getCatIds(catNms=[COCO_name])[0];

def COCOId_to_COCONames(coco, COCO_id):
  cats = coco.loadCats(coco.getCatIds())
  for c in cats:
    if c['id'] == COCO_id:
      return c['name']
  return -1

# Function to convert COCO format to YOLO
def convert_coco_to_yolo(coco, img_ids, output_labels_dir):
    for img_id in img_ids:
        img_info = coco.loadImgs([img_id])[0]
        ann_ids = coco.getAnnIds(imgIds=[img_id])
        annotations = coco.loadAnns(ann_ids)

        # Prepare YOLO format label
        label_file_path = Path(output_labels_dir + img_info['file_name'])
        label_file = label_file_path.with_suffix(".txt")
        with open(label_file, 'w+') as f:
            for ann in annotations:
                category_id = ann['category_id']
                if not category_id in catIds:
                  continue
                bbox = ann['bbox']
                segmentation = ann['segmentation']

                x = bbox[0]
                y = bbox[1]
                w = bbox[2]
                h = bbox[3]

                # Finding midpoints
                x_centre = (x + (x+w))/2
                y_centre = (y + (y+h))/2

                # Normalization
                x_centre = x_centre / img_info['width']
                y_centre = y_centre / img_info['height']
                w = w / img_info['width']
                h = h / img_info['height']

                # Limiting upto fix number of decimal places
                x_centre = format(x_centre, '.6f')
                y_centre = format(y_centre, '.6f')
                w = format(w, '.6f')
                h = format(h, '.6f')

                # Since YOLO train wants cat from 0 to N, we have to remap
                new_category_id = my_categories[COCOId_to_COCONames(cocoTrain, category_id)]

                f.write(f"{new_category_id} {x_centre} {y_centre} {w} {h} ")
                #If segmentation exists, append the polygons (optional)
                if isinstance(segmentation, list):  # Polygons

                  segmentation_points_list = []
                  for segmentation in ann['segmentation']:

                    # segmentation_points = [str(\
                    #   float(point) / (img_info['width']-1) if i % 2 == 0 \
                    #   else float(point) / (img_info['height']-1)) \
                    #   for i, point in enumerate(segmentation)]

                    #cap to 1.0 so yolo does not complain
                    segmentation_points = [
                        str(min(float(point) / (img_info['width'] - 1), 1.0)) if i % 2 == 0
                        else str(min(float(point) / (img_info['height'] - 1), 1.0))
                        for i, point in enumerate(segmentation)
                    ]
                    
                    segmentation_points_list.append(' '.join(segmentation_points))
                    segmentation_points_string = ' '.join(segmentation_points_list)
                    line = '{} '.format(segmentation_points_string)
                    f.write(line)
                    segmentation_points_list.clear()
                  f.write(f"\n")
                else:
                  f.write(f"\n")


def get_ids(coco, cat_names, N):
  catIds = coco.getCatIds(cat_names);
  imgIds = []
  for cat in catIds:
    imgIdsAll = coco.getImgIds(catIds=cat)
    to_select = N
    if len(imgIdsAll) < N:
      to_select = len(imgIdsAll)
      print(f"WARNING: cat {cat} has only {len(imgIdsAll)}, when {N} requested")

    imgIdsSelection = np.random.choice(imgIdsAll, size=to_select, replace=False)

    imgIds += imgIdsSelection.tolist()

  #remove duplicates
  imgIds_no_dup = list(dict.fromkeys(imgIds))

  return imgIds_no_dup


def download_copy_images(imgs_folder, output_img_dir, coco, img_ids):
  imgs_info = coco.loadImgs(img_ids)
  for img_info in imgs_info:
    #check if file is already in img folder
    if not os.path.isfile(imgs_folder + img_info['file_name']):
      # Save the image into a local folder
      img_data = requests.get(img_info['coco_url']).content
      with open(imgs_folder + img_info['file_name'], 'wb') as handler:
          handler.write(img_data)

    shutil.copyfile(imgs_folder + img_info['file_name'], output_img_dir + img_info['file_name'])

def imageName_to_cocoId(coco, imageName):
  imgs_info = coco.loadImgs(coco.getImgIds())
  for img_info in imgs_info:
    if img_info['file_name'] == imageName:
      return img_info['id']
  return -1

def job(img_info, imgs_folder, output_img_dir):
  #check if file is already in img folder
  if not os.path.isfile(imgs_folder + img_info['file_name']):
    # Save the image into a local folder
    img_data = requests.get(img_info['coco_url']).content
    with open(imgs_folder + img_info['file_name'], 'wb') as handler:
        handler.write(img_data)
  shutil.copyfile(imgs_folder + img_info['file_name'], output_img_dir + img_info['file_name'])

def download_copy_images_pool(imgs_folder, output_img_dir, coco, img_ids, pool_count=5):
  pool = Pool(pool_count)
  partial_support = partial(job, imgs_folder = imgs_folder, output_img_dir = output_img_dir)
  imgs_info = coco.loadImgs(img_ids)
  bar = Bar('Downloading and copying...', max=len(imgs_info))
  for i in pool.imap(partial_support, imgs_info):
    bar.next()
  bar.finish()
  pool.close()
  pool.join()


In [ ]:
imageName_to_cocoId(cocoTrain, "000000033931.jpg")

test_ids = get_ids(cocoTrain, ["car"], 60)

download_copy_images_pool(img_folder, "/content/test/", cocoTrain, test_ids)

img_folder = "/content/imgs/"


## Params

In [ ]:
my_categories = {
    "person": 0,
    "sports ball": 1,
    "bottle": 2,
    "wine glass": 3,
    "cup": 4,
    "fork": 5,
    "knife": 6,
    "spoon": 7,
    "bowl": 8,
    "banana": 9,
    "apple": 10,
    "sandwich": 11,
    "orange": 12,
    "carrot": 13,
    "book": 14,
    "clock": 15,
    "vase": 16,
    "scissors": 17,
    "toothbrush": 18,
}

#laser is 309/76 (80/20)

#for each category, it is assured that AT LEAST this number of occurencies exist
# (because on same image multiple categories can be present, and also multiple occurencies
# of same category)
N_sample_train = 400
N_sample_valid = 100

output_name = "./coco_subset"


In [ ]:
# get all images containing given categories, select at random

id_train = get_ids(cocoTrain, list(my_categories.keys()), N_sample_train)
id_valid = get_ids(cocoVal, list(my_categories.keys()), N_sample_valid)

#add to valid set from the non selected among the train
id_valid_from_train = []
catIds = cocoTrain.getCatIds(list(my_categories.keys()));
for cat in catIds:
    imgIdsCat = cocoVal.getImgIds(catIds=cat)
    if len(imgIdsCat) < N_sample_valid:
      print(len(imgIdsCat))
      ids = cocoTrain.getImgIds(catIds=cat)
      i = 0
      for id in ids:
        if id not in id_train:
          id_valid_from_train.append(id)
          i = i+1
          if (i + len(imgIdsCat)) >= N_sample_valid:
            print(f"cat {cat} had only {len(imgIdsCat)}, added {i}")
            break

id_valid_from_train = list(dict.fromkeys(id_valid_from_train))

#remove from train things that are in valid, this should never happen, but who knows
print(len(id_train))
id_train = [i for i in id_train if i not in id_valid]
id_train = [i for i in id_train if i not in id_valid_from_train]
print(len(id_train))

print(f"Selected {len(id_train)} for train and {len(id_valid)} for validation, plus {len(id_valid_from_train)} from train to extend validation")

In [ ]:
selected_images = cocoTrain.loadImgs(id_train)

img = selected_images[8]
I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I)
plt.show()

plt.imshow(I); plt.axis('off')
annIds = cocoTrain.getAnnIds(imgIds=img['id'], iscrowd=None)
anns = cocoTrain.loadAnns(annIds)
cocoTrain.showAnns(anns)
print(anns)

In [ ]:
#append timestamp to output dir
output_dir = output_name + "_" + str(int(time.time()))

# Create output directories if they don't exist
Path(output_dir+"/train/labels/").mkdir(parents=True, exist_ok=True)
Path(output_dir+"/train/images/").mkdir(parents=True, exist_ok=True)
Path(output_dir+"/valid/labels/").mkdir(parents=True, exist_ok=True)
Path(output_dir+"/valid/images/").mkdir(parents=True, exist_ok=True)

# Convert annotations
convert_coco_to_yolo(cocoTrain, id_train, output_dir+"/train/labels/")
convert_coco_to_yolo(cocoVal, id_valid, output_dir+"/valid/labels/")
convert_coco_to_yolo(cocoTrain, id_valid_from_train, output_dir+"/valid/labels/")

download_copy_images_pool(img_folder, output_dir+"/train/images/", cocoTrain, id_train)
download_copy_images_pool(img_folder, output_dir+"/valid/images/", cocoVal, id_valid)
download_copy_images_pool(img_folder, output_dir+"/valid/images/", cocoTrain, id_valid_from_train)


# Create yaml
yaml_to_dump = {
    "train": "./train/images",
    "val": "./valid/images",
    "nc": len(my_categories),
    "names": dict(zip(my_categories.values(), my_categories.keys()))
}

#dump to data.yaml
with open(output_dir + "/data.yaml", "w+") as outfile:
    yaml.dump(yaml_to_dump, outfile, default_flow_style=False)



Download

In [ ]:
import shutil
shutil.make_archive(output_dir, 'zip', output_dir)

from google.colab import files
files.download(output_dir + ".zip")

**VISUALIZA MASK to see if it is correct**

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw,ImageFont
import random
from matplotlib import pyplot as plt

image_path = "./"+ output_dir +"/train/images/000000138385.jpg"  # path of the image, change it
annotation_path = "./"+ output_dir +"/train/labels/000000138385.txt"  # path of the annotation text file, change it

# The Helper functions below assume that the image size is (640,640).Hence resizing the image.
#Open the image
img = Image.open(image_path)
#Resize the image to 640 by 640
img = img.resize((640, 640))
#if you want then you can save the resized image by img.save('resized_image.jpg')

# <--------- Helper functions starts here ----------------------------------->
def maskVisualize(image,mask):
  fontsize = 18
  f, ax = plt.subplots(2, 1, figsize=(8, 8))
  ax[0].imshow(image)
  ax[1].imshow(mask)

#Define the boundary coordinates as a list of (x, y) tuples
'''
def draw_points_on_image(image, points):
  #resize image to 640*640
  resimg = cv2.resize(image, (640,640))
  #iterate for each mask
  for mask in points:
    #Draw each point on the image
    for point in mask:
        cv2.circle(resimg, tuple(point), 1, (0,0,255), -1,)
  #Display the image
  cv2_imshow(resimg)
'''
#convert the mask from the txt file(annotation_path is path of txt file) to array of points making that mask.
def generate_points(annotation_path=''):
  labels=[] # this will store labels
  #we are assuming that the image is of dimension (640,640). then you have annotated it.
  with open(annotation_path, "r") as file:
    points=[]
    for line in file:
      label,bbox,lis=line.split()[0],line.split()[1:5], line.split()[5:]
      labels.append(label)
      lis=list(map(float,lis))
      for i in range(len(lis)):
        lis[i]=int(lis[i]*640)
      newlis=[]
      i=0
      while(i<len(lis)):
        #appendint the coordinates as a tuple (x,y)
        newlis.append((lis[i],lis[i+1]))
        i+=2
      points.append(newlis)
    return labels,points


#the below function convert the boundary coordinates to mask array (it shows mask if you pass 1 at show)
#the mask array is required when we want to augument the mask also using albumentation
def convert_boundary_to_mask_array(labels,points, show=0):
  #Create a new image with the same size as the desired mask
  mask = Image.new("L", (640, 640), 0)
  draw = ImageDraw.Draw(mask)
  for i,boundary_coords in enumerate(points):
    #boundary_coords represent boundary of one polygon
    #Draw the boundary on the mask image
    if len(boundary_coords) == 0:
      continue
    draw.polygon(boundary_coords,fill=1)
    #Also put the label as text
    #Compute the centroid of the polygon
    centroid_x = sum(x for x, _ in boundary_coords) / len(boundary_coords)
    centroid_y = sum(y for _, y in boundary_coords) / len(boundary_coords)
    centroid = (int(centroid_x), int(centroid_y))
    #Write the name at the centroid
    text = str(labels[i])
    #Write the label at the centroid
    font = ImageFont.load_default()
    font.size = 30
    #text_w, text_h = draw.textsize(text, font=font)
    text_w = draw.textlength(text, font=font)
    text_h = font.size * 1
    text_pos = (centroid[0] - text_w/2, centroid[1] - text_h/2)
    draw.text(text_pos, text, font=font, fill='black')
  #Convert the mask image to a numpy array
  mask_array = np.array(mask)*255
  #Show the mask image
  if(show==1):
    #Image.fromarray(mask_array).show()
    cv2_imshow(mask_array)
  return mask_array

#function that takes mask path (yolov8 seg txt file) and return mask of an image (shape of mask == shape of image)
def generate_mask(annotation_path='',show=0):
  #pass show=1 for showing the generated mask
  #firstly we generate the points (coordinates) from the annotations
  labels,points=generate_points(annotation_path)
  #once we get the points we will now generate the mask image from these points (binary mask image (black/white))
  #mask is represented by white and ground is represented as black
  mask_array=convert_boundary_to_mask_array(labels,points,show)
  return mask_array
# <---------- Helper Functions Ends here ------------------------------------------------------------->

mask_array=generate_mask(annotation_path=annotation_path,show=0)
maskVisualize(np.array(img),mask_array)


In [ ]:
##Show bboxes
img = cv2.imread(image_path)
dh, dw, _ = img.shape

with open(annotation_path, "r") as file:
  points=[]
  for dt in file:
    label,bbox=dt.split()[0],dt.split()[1:5]

    # Split string to float
    x = float(bbox[0])
    y = float(bbox[1])
    w = float(bbox[2])
    h = float(bbox[3])

    # Taken from https://github.com/pjreddie/darknet/blob/810d7f797bdb2f021dbe65d2524c2ff6b8ab5c8b/src/image.c#L283-L291
    # via https://stackoverflow.com/questions/44544471/how-to-get-the-coordinates-of-the-bounding-box-in-yolo-object-detection#comment102178409_44592380
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)

    if l < 0:
        l = 0
    if r > dw - 1:
        r = dw - 1
    if t < 0:
        t = 0
    if b > dh - 1:
        b = dh - 1

    cv2.rectangle(img, (l, t), (r, b), (0, 0, 255), 1)
    cv2.putText(img, label, (l, t), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)


plt.imshow(img)
plt.show()

In [ ]:
!rm -r prova*